In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io as sio
import seaborn as sns
import statsmodels.api as sm
import funcs as func
import matplotlib.pyplot as plt
import sys
import pdb
import glob
import pickle
import dill

import sys
sys.path.append('/home/jyotika/Utils/hddm/')

import hddm

data_dir = "./Data/Y_maze/Decision_times/"
data_target_dir = "./Data/processed_data/Y_maze/data_with_animal_ids/"
figure_dir = "./Figures/Y_maze/"

plt.rcParams["figure.facecolor"] = "w"

In [ ]:
all_conflicts = pd.read_csv(data_target_dir+"all_experiments_df_with_DTs.csv")
all_conflicts = all_conflicts.loc[all_conflicts["condition"]=="Control"]
# all_conflicts["Day"] = [ int(x/5)  for x in all_conflicts["session"]]

In [ ]:
# g1 = sns.catplot(x="trial_num",y="RT(ms)",col="conflict + volatility",data=all_conflicts,col_wrap=3,kind='point',hue='block_num',n_boot=100,sharey=False,sharex=False )

In [ ]:
all_conflicts

In [ ]:
for grp in all_conflicts.groupby(["condition","conflict","volatility","session"]):
    
    g1 = sns.relplot(x="block_num",y="RT(ms)",data=grp[1],hue="animal_id",kind='line')
    

In [ ]:
sess5 = all_conflicts.loc[(all_conflicts["session"]>4)&(all_conflicts["session"]<9)&(all_conflicts["block_num"]<50)&(all_conflicts["volatility"]=="Low")]
sess5["block_group(of 10)"] = [ int(x/20)  for x in sess5["block_num"]]
sess5["(session,block)"] = [ (y,int(x/20))  for x,y in zip(sess5["block_num"],sess5["session"]) ]

In [ ]:
hue_order = np.unique(sess0["(session,block)"].sort_values())

In [ ]:
g1 = sns.relplot(x="block_num",y="RT(ms)",col="conflict",data=sess5,col_wrap=1,kind='line',n_boot=100,facet_kws={'sharex':False},palette='tab10',col_order=["No","Low", "High"] ,height=5,aspect=4,hue="block")
for ax in g1.axes:
    xlims = ax.get_xlim()
    ax.hlines(y=500,xmin=xlims[0],xmax=xlims[1],color='k',ls='dashed',lw=2.5)
    ax.set_xlim(xlims[0],xlims[1])

In [ ]:
sns.catplot(x="conflict",y="RT(ms)",data=all_conflicts,order=["No","Low","High"],col="volatility",kind='point',hue='session')

In [ ]:
np.unique(sess0["(session,block)"].sort_values())

In [ ]:
block_change_aligned_df = pd.DataFrame()
for grp in all_conflicts.groupby(["conflict + volatility","Day","session"]):
    print(grp[0])
    dat_slice = grp[1].copy()
    dat_slice["block_change"] = (grp[1]["block_num"]!=grp[1]["block_num"].shift()).astype(int)
    ind_block_change = np.where(dat_slice["block_change"])[0]
    print(ind_block_change)
    for x in ind_block_change[1:]:
        if dat_slice.iloc[x-1]["block"] == "left" and dat_slice.iloc[x]["block"] == "right":
            bc_type = "left->right"
        elif dat_slice.iloc[x-1]["block"] == "right" and dat_slice.iloc[x]["block"] == "left":
            bc_type = "right->left"
        for i in np.arange(x-2,x+10):
            if i > np.max(ind_block_change):
                continue
            #print(i)
            sub_slice = dat_slice.iloc[i]
            sub_slice["trials_from_change_point"] = i-x
            sub_slice["block_change_type"] = bc_type
            block_change_aligned_df = block_change_aligned_df.append(sub_slice)

In [ ]:
block_change_aligned_df = block_change_aligned_df.reset_index(drop=True)

In [ ]:
block_change_aligned_df.iloc[np.where(block_change_aligned_df.index.duplicated())]

In [ ]:
block_change_aligned_df.to_csv(data_target_dir+"block_change_aligned_df.csv")

In [ ]:
col_order = [ conf+" - "+vol  for vol in ["Low","High"] for conf in ["No","Low","High"]]
col_order

In [ ]:
block_change_aligned_df["session_group"] = [ int(x/3)  for x in block_change_aligned_df["session"]]
block_change_aligned_df["sg + bc_type"] = block_change_aligned_df["session_group"].astype(str)+" - "+block_change_aligned_df["block_change_type"]
block_change_aligned_df["block_group"] = [ int(x/5)  for x in block_change_aligned_df["block_num"]]
block_change_aligned_df["error_rate"] = 1- block_change_aligned_df["optimal_action"].values

# sess = block_change_aligned_df.loc[(block_change_aligned_df["session"]<20)&(block_change_aligned_df["session"]>0)]
# sess = block_change_aligned_df.loc[block_change_aligned_df["session"]==2] # very good session 2,5
# sess = block_change_aligned_df.loc[block_change_aligned_df["session"]==5]
sess_no = block_change_aligned_df.loc[(block_change_aligned_df["conflict"]== "No")&(block_change_aligned_df["volatility"]=="Low")]
sess_low = block_change_aligned_df.loc[(block_change_aligned_df["conflict"]== "Low")&(block_change_aligned_df["volatility"]=="Low")]
sess_high = block_change_aligned_df.loc[(block_change_aligned_df["conflict"]== "High")&(block_change_aligned_df["volatility"]=="Low")]

In [ ]:
sess

In [ ]:
# sns.relplot(x="trials_from_change_point",y="RT(ms)",data=sess,col="conflict + volatility",col_wrap=3,kind='line',hue='session_group',palette='magma',col_order=col_order,lw=3.5,facet_kws={'sharey':True})
# sns.relplot(x="trials_from_change_point",y="RT(ms)",data=sess,col="conflict",col_wrap=3,kind='line',hue='session_group',palette='magma',lw=3.5,facet_kws={'sharey':True},col_order=["No","Low","High"])
g1 = sns.relplot(x="trials_from_change_point",y="RT(ms)",data=sess_no,col="Day",col_wrap=3,kind='line',hue='block_change_type',palette='magma',lw=3.5,facet_kws={'sharey':True})
for ax in g1.axes:
    ax.set_ylim(400,3000)


In [ ]:
g1 = sns.relplot(x="trials_from_change_point",y="RT(ms)",data=sess_low,col="Day",col_wrap=3,kind='line',hue='block_change_type',palette='magma',lw=3.5,facet_kws={'sharey':True})
for ax in g1.axes:
    ax.set_ylim(400,3000)


In [ ]:
g1 = sns.relplot(x="trials_from_change_point",y="RT(ms)",data=sess_high,col="Day",col_wrap=3,kind='line',hue='block_change_type',palette='magma',lw=3.5,facet_kws={'sharey':True})
for ax in g1.axes:
    ax.set_ylim(400,3000)


In [ ]:
# sns.catplot(x="conflict",y="RT(ms)",data=sess,col="volatility",order=["No","Low","High"],kind='point',hue='condition')
sns.catplot(x="conflict",y="RT(ms)",data=sess,order=["No","Low","High"],kind='point',hue='block_change_type',col_order=["No","Low","High"])

In [ ]:
# sns.relplot(x="trials_from_change_point",y="error_rate",data=sess,col="conflict + volatility",col_wrap=3,kind='line',hue='session_group',palette='magma',col_order=col_order,lw=3.5,facet_kws={'sharey':True})
g1 = sns.relplot(x="trials_from_change_point",y="error_rate",data=sess_no,col="Day",col_wrap=3,kind='line',hue='block_change_type',palette='magma',lw=3.5,facet_kws={'sharey':True},hue_order=["left->right","right->left"])
for ax in g1.axes:
    ax.set_ylim(0,1.0)

In [ ]:
g1 = sns.relplot(x="trials_from_change_point",y="error_rate",data=sess_low,col="Day",col_wrap=3,kind='line',hue='block_change_type',palette='magma',lw=3.5,facet_kws={'sharey':True},hue_order=["left->right","right->left"])
for ax in g1.axes:
    ax.set_ylim(0,1.0)

In [ ]:
g1 = sns.relplot(x="trials_from_change_point",y="error_rate",data=sess_high,col="Day",col_wrap=3,kind='line',hue='block_change_type',palette='magma',lw=3.5,facet_kws={'sharey':True},hue_order=["left->right","right->left"])
for ax in g1.axes:
    ax.set_ylim(0,1.0)

In [ ]:
sns.pointplot(x="conflict",y="optimal_action",data=sess,order=["No","Low","High"],hue="sg + bc_type")

In [ ]:
temp = pd.DataFrame(all_conflicts.groupby(["conflict", "volatility","Day","session","block_num","block"])["RT(ms)"].mean())

In [ ]:
temp1 = pd.DataFrame(all_conflicts.groupby(["conflict", "volatility","Day","session","block_num","block"])["optimal_action"].mean())

In [ ]:
temp1

In [ ]:
temp.index.get_level_values(0)


In [ ]:
average_rt = pd.DataFrame()
average_rt["conflict"] = temp.index.get_level_values(0)
average_rt["volatility"] = temp.index.get_level_values(1)
average_rt["day"] = temp.index.get_level_values(2)
average_rt["session"] = temp.index.get_level_values(3)
average_rt["block_num"] = temp.index.get_level_values(4)
average_rt["block"] = temp.index.get_level_values(5)
average_rt["RT(ms)"] = temp["RT(ms)"].values



In [ ]:
average_err = pd.DataFrame()
average_err["conflict"] = temp1.index.get_level_values(0)
average_err["volatility"] = temp1.index.get_level_values(1)
average_err["day"] = temp1.index.get_level_values(2)
average_err["session"] = temp1.index.get_level_values(3)
average_err["block_num"] = temp1.index.get_level_values(4)
average_err["block"] = temp1.index.get_level_values(5)
average_err["error_rate"] =  1- temp1["optimal_action"].values


In [ ]:
average_rt_low = average_rt.loc[average_rt["volatility"]=="Low"]
average_rt_high = average_rt.loc[average_rt["volatility"]=="High"]


In [ ]:
average_err_low = average_err.loc[average_err["volatility"]=="Low"]
average_err_high = average_err.loc[average_err["volatility"]=="High"]


In [ ]:
# sns.relplot(x="block_num",y="RT(ms)",data=average_rt_low.loc[average_rt_low["session"]<5],col="conflict",hue="session",kind='line',lw=3.5,col_order=["No","Low","High"])

In [ ]:
g1 = sns.relplot(x="session",y="RT(ms)",data=average_rt_low,col="conflict",kind='line',lw=3.5,col_order=["No","Low","High"],hue="block")
g1.savefig(figure_dir+"average_RT_sessions_low_volatility.png")

In [ ]:
g1 = sns.relplot(x="session",y="RT(ms)",data=average_rt_high,col="conflict",kind='line',lw=3.5,col_order=["No","Low","High"],hue="block",facet_kws={'sharey':False})
g1.savefig(figure_dir+"average_RT_sessions_high_volatility.png")

In [ ]:
g1 = sns.relplot(x="session",y="error_rate",data=average_err_low,col="conflict",kind='line',lw=3.5,col_order=["No","Low","High"],hue='block')
g1.savefig(figure_dir+"error_rate_session_wise_low_volatility.png")


In [ ]:
sns.relplot(x="session",y="RT(ms)",data=average_rt_high,col="conflict",kind='line',lw=3.5,col_order=["No","Low","High"],hue="block",facet_kws={'sharey':False})

In [ ]:
sess0_4 = block_change_aligned_df.loc[(block_change_aligned_df["session"]<5)&(block_change_aligned_df["volatility"]=="Low")]
sns.relplot(x="trials_from_change_point",y="RT(ms)",data=sess0_4,col="conflict",col_wrap=3,kind='line',hue='sg + bc_type',palette='magma',lw=3.5,facet_kws={'sharey':True},col_order=["No","Low","High"])

In [ ]:
g1 = sns.relplot(x="trials_from_change_point",y="error_rate",data=sess0_4,col="conflict",col_wrap=3,kind='line',hue='sg + bc_type',palette='magma',lw=3.5,facet_kws={'sharey':True},col_order=["No","Low","High"])
g1.savefig(figure_dir+"error_rates_low_volatility.png")

In [ ]:
average_rt_low

In [ ]:
average_err_low

In [ ]:
average_err_rt = average_err_low.merge(average_rt_low,on=["session","block_num","conflict","block","day"])

In [ ]:
average_err_rt

In [ ]:
sns.jointplot(x="RT(ms)",y="error_rate",data=average_err_rt.loc[average_err_rt["conflict"]=="No"],hue="day",kind='kde',alpha=0.5,palette='tab10')

In [ ]:
sns.jointplot(x="RT(ms)",y="error_rate",data=average_err_rt.loc[average_err_rt["conflict"]=="Low"],hue="day",kind='kde',alpha=0.5,palette='tab10')

In [ ]:
sns.jointplot(x="RT(ms)",y="error_rate",data=average_err_rt.loc[average_err_rt["conflict"]=="High"],hue="day",kind='kde',alpha=0.5,palette='tab10')

In [ ]:
sns.lmplot(x="RT(ms)",y="error_rate",data=average_err_rt.loc[(average_err_rt["conflict"]=="High")&(average_err_rt["day"]<2)],hue="day")


In [ ]:
sns.lmplot(x="RT(ms)",y="error_rate",data=average_err_rt.loc[(average_err_rt["conflict"]=="Low")&(average_err_rt["day"]<2)],hue="day")

In [ ]:
sns.lmplot(x="RT(ms)",y="error_rate",data=average_err_rt.loc[(average_err_rt["conflict"]=="No")&(average_err_rt["day"]<2)],hue="day")